# Защита персональных данных клиентов

Нам поступила задача защитить данные клиентов страховой компании «Хоть потоп». В процессе работы необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. В итоге потребуется обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Для достижения поставленной цели требуется выполнить задачи:
- установить, изменится ли качество линейной регрессии при умножении признаков на обпатимую матрицу;
- предложить алгоритм преобразования данных для решения задачи;
- запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применить метрику R2.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

In [1]:
# Импортируем необходимые библиотеки для решения поставленной задачи
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
# Загрузим данные
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
# Посмотрим на первые 10 строк таблицы
df.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [4]:
# Посмотрим на общую информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Мы видим, что пропусков в таблице нет. В ней 5000 объектов и 5 признаков.

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [5]:
# Проверим данные на наличие дубликатов
print('В таблице доля дубликтов составляет:', df.duplicated().sum() / len(df) * 100, '%.')

В таблице доля дубликтов составляет: 3.06 %.


Так как доля их мала, то для дальнейшего анализа целесообразно будет их удалить.

In [6]:
df = df.drop_duplicates().reset_index(drop=True)

## Умножение матриц

Создадим модель линейной регрессии и обучим ее.

In [7]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print('Значение r2 метрики:', r2_score(target, predictions))

Значение r2 метрики: 0.4302010044852067


In [8]:
# Создадим случайную матрицу и проверим ее на обратимость
random_matrix = np.random.normal(3, 2.5, size=(4, 4))
try:
    np.linalg.inv(random_matrix)
    print('Матрица обратимая')
except:
    np.linalg.inv(random_matrix)
    print('Ошибка - матрица необратимая!')

Матрица обратимая


In [9]:
# Умножим признаки на обратимую матрицу
features_new = features.dot(random_matrix)

In [10]:
# Обучим модель на новых признаках и посчитаем r2
model = LinearRegression()
model.fit(features_new, target)
predictions = model.predict(features_new)
print('Значение r2 метрики:', r2_score(target, predictions))

Значение r2 метрики: 0.43020100448520626


Мы видим, что значение r2 практически не изменилось.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество линейной регрессии не изменится.

**Обоснование:** связь между признаками линейная, при умножении фичей на обратимую матрицу добавляются коэффициенты, что в итоге это не влияет на результат.

## Алгоритм преобразования

**Алгоритм**

Предложим алгоритм преобразования данных для решения задачи.
1. Создание случайной квадратной матрицы Y.
2. Проверка матрицы на обратимость.
3. Получение матрицы преобразованных признаков Z = XY.
4. Применение алгоритма на преобразованных признаках

**Обоснование**

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Заменим матрицу X на матрицу Z:

$$
Z = XP
$$

Подставим матрицу Z в формулу обучения вместо X:

$$
w' = (Z^T Z)^{-1} Z^T y
$$

Заменим матрицу Z на XP:

$$
w' = ((XP)^T (XP))^{-1} (XP)^T y
$$

Далее воспользуемся свойствами матрицы:

$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$

Преобразуем формулу с учетом указанного свойства:

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
$$
w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Подставим значение w' в формулу предсказаний:

$$
y' = w' Z = Z^{-1} y' Z = y'
$$

**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T \tag{1}
$$
$$
(AB)^{-1} = B^{-1} A^{-1} \tag{2}
$$
$$
A A^{-1} = A^{-1} A = E  \tag{3}
$$
$$
AE = EA = A \tag{4}
$$
Доказательство:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
$$
w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = ((P^T)^{-1} P^T) (X^T X)^{-1} P^{-1} X^T y
$$
$$
w' = E (X^T X)^{-1} P^{-1} X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$
Подставим значение w' в формулу предсказания:
$$
a = XP P^{-1} (X^T X)^{-1} X^T y = EX (X^T X)^{-1} X^T y = X (X^T X)^{-1} X^T y
$$

Таким образом, было доказано, что умножение признаков на обратимую матрицу не меняет значения предсказаний, а значит, качество линейной регрессии не поменяется.

## Проверка алгоритма

Перед проверкой алгоритма создадим необходимые перменные.ё

In [17]:
X = features
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
y = target
P = np.random.normal(3, 2.5, size=(5, 5))

In [18]:
# Проверим случайную матрицу P на обратимость
try:
    np.linalg.inv(P)
    print('Матрица обратимая')
except:
    np.linalg.inv(P)
    print('Ошибка - матрица необратимая!')

Матрица обратимая


In [19]:
# Получим матрицу преобразованных признаков
Z = X @ P

In [20]:
# Создадим функцию для разбивки выборки на обучающую и тестовую
def test_split(features):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)
    return X_train, X_test, y_train, y_test

In [21]:
# Разделим выборки на обучающую и тестовую
X_train, X_test, y_train, y_test = test_split(X)
Z_train, Z_test, y_train, y_test = test_split(Z)

In [22]:
# Создадим функцию для обучения линейной регрессии
def lr(features):
    model = LinearRegression()
    model.fit(features, y_train)
    predictions = model.predict(X_test)
    return r2_score(y_test, predictions)

In [23]:
# Применим функцию для вычисления r2 метрики для модели с начальными признаками и преобразованными
print('r2 для модели с начальными признаками:', lr(X_train))
print('r2 для модели с преобразованными признаками:', lr(Z_train))

r2 для модели с начальными признаками: 0.42307727492148095
r2 для модели с преобразованными признаками: 0.42307727492148095


Значение метрики r2 не изменилось после преобразования начальных признаков.

**Вывод**

Таким образом, в процессе решения поставленных задач было доказано неизменяемость метрики качества для линейной модели регрессии при умножении признаков выборки на обратимую матрицу. Также был составлен алгоритм преобразования данных для решения задачи. В завершение даннных алгоритм был запрограммирован и проверен на наших данных, в результате чего значение метрики r2 не изменилось.